# UniHPF Style Linearization Example

## Synthetic Data
For this example, we'll build on the local, sample data tutorial in the `sample_data/examine_synthetic_data.ipynb` notebook. We'll highlight some of the differences from that tutorial first, then skip to the relevant new, UniHPF components. The notable difference in setup is that there is now a `medications.csv` file that we'll use to showcase a more complex data structure

In [1]:
!ls --color -lah raw

total 56M
drwxrwxr-x 2 mmd mmd 4.0K Aug 23 13:49 .
drwxrwxr-x 5 mmd mmd 4.0K Aug 23 14:01 ..
-rw-rw-r-- 1 mmd mmd 3.6M Aug 23 13:57 admit_vitals.csv
-rw-rw-r-- 1 mmd mmd  52M Aug 23 13:57 labs.csv
-rw-rw-r-- 1 mmd mmd 132K Aug 23 13:57 medications.csv
-rw-rw-r-- 1 mmd mmd 4.2K Aug 23 13:57 subjects.csv


In [2]:
!csvlook -d "," --max-rows=5 raw/medications.csv

|       MRN | timestamp           | name     | dose | frequency |         duration | generic_name   |
| --------- | ------------------- | -------- | ---- | --------- | ---------------- | -------------- |
|   415,881 | 08:03:23-2010-06-09 | motrin   |  800 | 2x/day    |  5 days, 0:00:00 | Ibuprofen      |
|    72,293 | 23:02:03-2010-05-08 | Tylenol  |  500 | 1x/day    |   1 day, 0:00:00 | Acetaminophen  |
| 1,447,535 | 21:46:21-2010-07-22 | Advil    |  700 | 1x/day    |  7 days, 0:00:00 | Ibuprofen      |
|   538,150 | 12:54:33-2010-04-29 | Tylenol  |  500 | 4x/day    |   1 day, 0:00:00 | Acetaminophen  |
|   928,262 | 06:56:17-2010-08-26 | Benadryl |    0 | 1x/day    | 11 days, 0:00:00 | Diphenydramine |
|       ... | ...                 | ...      |  ... | ...       |              ... | ...            |


## Processing Synthetic Data with ESGPT

Much like before, we'll parse this with ESGPT, though this time we'll turn off normalization, outlier detection, not drop any measurements due to frequency, and retain some extra columns for the medication source. Here's the config we'll use for that:

In [3]:
!cat dataset_unihpf.yaml

defaults:
  - dataset_base
  - _self_

# So that it can be run multiple times without issue.
do_overwrite: True

cohort_name: "unihpf_sample"
subject_id_col: "MRN"
raw_data_dir: "./sample_data/raw/"
save_dir: "./sample_data/processed/${cohort_name}"

DL_chunk_size: null

inputs:
  subjects:
    input_df: "${raw_data_dir}/subjects.csv"
  admissions:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    start_ts_col: "admit_date"
    end_ts_col: "disch_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
    event_type: ["OUTPATIENT_VISIT", "ADMISSION", "DISCHARGE"]
  vitals:
    input_df: "${raw_data_dir}/admit_vitals.csv"
    ts_col: "vitals_date"
    ts_format: "%m/%d/%Y, %H:%M:%S"
  labs:
    input_df: "${raw_data_dir}/labs.csv"
    ts_col: "timestamp"
    ts_format: "%H:%M:%S-%Y-%m-%d"
  medications:
    input_df: "${raw_data_dir}/medications.csv"
    ts_col: "timestamp"
    ts_format: "%H:%M:%S-%Y-%m-%d"
    columns: {"name": "medication"}


measurements:
  static:
    single_label_classificat

There are two major modifications of note. First, beyond just adding the `medications` input source, we also add a `medication` measurement which has the additional `modifiers` key, which tracks other columns that should be leveraged there:
```yaml
      medications:
        - name: name
          modifiers: 
            - [dose, "float"]
            - [frequency, "categorical"]
            - [duration, "categorical"]
            - [generic_name, "categorical"]
```
`modifiers` takes a list of pairs of column names and column import types which should also be read.

In addition, we've set many other parameters to `null` to turn those components of the pre-processing pipeline off:
```yaml
outlier_detector_config: null
normalizer_config: null
min_valid_vocab_element_observations: null
min_valid_column_observations: null
min_true_float_frequency: null
min_unique_numerical_observations: null
...
agg_by_time_scale: null
```

Now, with that, we can run the pipeline with this command:
```bash
PYTHONPATH=$(pwd):$PYTHONPATH ./scripts/build_dataset.py \
	--config-path="$(pwd)/sample_data/" \
	--config-name=dataset_unihpf \
	"hydra.searchpath=[$(pwd)/configs]"
```

In [6]:
import subprocess

command = """\
PYTHONPATH=$(pwd):$PYTHONPATH ./scripts/build_dataset.py \
 --config-path="$(pwd)/sample_data/" \
 --config-name=dataset_unihpf \
 "hydra.searchpath=[$(pwd)/configs]" """

command_out = subprocess.run(command, cwd="../", shell=True, capture_output=True)
print(command_out.stdout.decode())

if command_out.returncode == 1:
    print("Command Errored!")

print(command_out.stderr.decode())

Empty new events dataframe of type OUTPATIENT_VISIT!




After we do so, the output will look very similar to the typical ESD pipeline:

In [7]:
!du -sh processed/unihpf_sample/

59M	processed/unihpf_sample/


In [8]:
!ls --color -lah -R processed/unihpf_sample/

processed/unihpf_sample/:
total 35M
drwxrwxr-x 5 mmd mmd 4.0K Aug 23 14:03 .
drwxrwxr-x 5 mmd mmd 4.0K Aug 23 14:03 ..
-rw-rw-r-- 1 mmd mmd 2.2K Aug 23 14:03 config.json
drwxrwxr-x 2 mmd mmd 4.0K Aug 23 14:03 DL_reps
-rw-rw-r-- 1 mmd mmd  13M Aug 23 14:03 dynamic_measurements_df.parquet
-rw-rw-r-- 1 mmd mmd 5.0K Aug 23 14:03 E.pkl
-rw-rw-r-- 1 mmd mmd  22M Aug 23 14:03 events_df.parquet
-rw-rw-r-- 1 mmd mmd 1.9K Aug 23 14:03 hydra_config.yaml
-rw-rw-r-- 1 mmd mmd 2.7K Aug 23 14:03 inferred_measurement_configs.json
drwxrwxr-x 2 mmd mmd 4.0K Aug 23 14:03 inferred_measurement_metadata
-rw-rw-r-- 1 mmd mmd 2.2K Aug 23 14:03 input_schema.json
drwxrwxr-x 3 mmd mmd 4.0K Aug 23 14:03 .logs
-rw-rw-r-- 1 mmd mmd 2.7K Aug 23 14:03 subjects_df.parquet
-rw-rw-r-- 1 mmd mmd  764 Aug 23 14:03 vocabulary_config.json

processed/unihpf_sample/DL_reps:
total 25M
drwxrwxr-x 2 mmd mmd 4.0K Aug 23 14:03 .
drwxrwxr-x 5 mmd mmd 4.0K Aug 23 14:03 ..
-rw-rw-r-- 1 mmd mmd 2.9M Aug 23 14:03 held_out_0.parquet
-rw

Inspecting the input schema, we can see that it is being instructed to pull the modifier columns:

In [9]:
import json
with open('processed/unihpf_sample/input_schema.json', mode='r') as f:
    cfg = json.load(f)
    dynamic_measurements = cfg['dynamic']
    for m in dynamic_measurements:
        if m['input_df'].endswith('medications.csv'):
            print(m['data_schema'][0])

{'name': ['medication', 'categorical'], 'dose': 'float', 'frequency': 'categorical', 'duration': 'categorical', 'generic_name': 'categorical'}


And, looking at the inferred measurement configs, we can further see that the modifiers list is still stored:

In [10]:
import json
with open('processed/unihpf_sample/inferred_measurement_configs.json', mode='r') as f:
    cfg = json.load(f)
    print(cfg['medication'])

{'name': 'medication', 'temporality': 'dynamic', 'modality': 'multi_label_classification', 'observation_frequency': 0.0013874774078514182, 'functor': None, 'vocabulary': {'vocabulary': ['UNK', 'Advil', 'Benadryl', 'Motrin', 'motrin', 'Tylenol'], 'obs_frequencies': [0.0, 0.2769423558897243, 0.23182957393483708, 0.18859649122807018, 0.17105263157894737, 0.13157894736842105]}, 'values_column': None, '_measurement_metadata': None, 'modifiers': ['dose', 'frequency', 'duration', 'generic_name']}


Note that even though `dose` was configured as a numeric, there is no metadata associated with it in the folder:

In [11]:
!ls processed/unihpf_sample/inferred_measurement_metadata/

age.csv  HR.csv  lab_name.csv  temp.csv


Further, given the config's instructions, there are no outlier or normalizer parameters learned for the other measurements:

In [12]:
!csvlook -d "," processed/unihpf_sample/inferred_measurement_metadata/age.csv

/usr/lib/python3/dist-packages/agate/utils.py:285: UnnamedColumnWarning: Column 0 has no name. Using "a".
| a             | age   |
| ------------- | ----- |
| value_type    | float |
| outlier_model |       |
| normalizer    |       |


Let's look at the actual dataset as well:

In [13]:
import sys

sys.path.append("../")

In [14]:
# Imports
import os, polars as pl
from pathlib import Path

from EventStream.data.dataset_polars import Dataset

In [15]:
dataset_dir = Path(os.getcwd()) / "processed/unihpf_sample"

With the dataset loaded, we can ask about the three dataframes we inspected above...

In [16]:
ESD = Dataset.load(dataset_dir)
display(ESD.subjects_df.head(3))
display(ESD.events_df.head(3))
display(ESD.dynamic_measurements_df.head(3))

Loading subjects from /home/mmd/Projects/EventStreamGPT/sample_data/processed/unihpf_sample/subjects_df.parquet...


subject_id,MRN,eye_color,dob
u8,cat,cat,datetime[μs]
0,"""310243""","""GREEN""",1981-07-28 00:00:00
1,"""384198""","""BROWN""",1985-04-15 00:00:00
2,"""520533""","""BROWN""",1979-04-15 00:00:00


Loading events from /home/mmd/Projects/EventStreamGPT/sample_data/processed/unihpf_sample/events_df.parquet...


event_id,subject_id,timestamp,event_type,age
u32,u8,datetime[μs],cat,f64
0,0,2010-06-24 13:23:00,"""ADMISSION&VITA…",28.907755
1,0,2010-06-24 13:46:14,"""LAB""",28.9078
2,0,2010-06-24 14:07:49,"""VITAL""",28.907841


Loading dynamic_measurements from /home/mmd/Projects/EventStreamGPT/sample_data/processed/unihpf_sample/dynamic_measurements_df.parquet...


measurement_id,department,HR,temp,lab_name,lab_value,medication,dose,frequency,duration,generic_name,event_id
u32,cat,f64,f64,str,f64,cat,f32,cat,cat,cat,u32
0,"""PULMONARY""",null,null,null,null,null,null,null,null,null,1116037
1,"""CARDIAC""",null,null,null,null,null,null,null,null,null,396335
2,"""PULMONARY""",null,null,null,null,null,null,null,null,null,1469231


We can see that the additional columns tagged have been extracted as well, and that lab values haven't been normalized.

In [17]:
display(ESD.dynamic_measurements_df.filter(pl.col('medication').is_not_null()).head(2))
display(ESD.dynamic_measurements_df.filter(pl.col('lab_name').is_not_null()).head(2))

measurement_id,department,HR,temp,lab_name,lab_value,medication,dose,frequency,duration,generic_name,event_id
u32,cat,f64,f64,str,f64,cat,f32,cat,cat,cat,u32
1474558,null,null,null,null,null,"""Benadryl""",100.0,"""1x/day""","""12 days""","""Diphenydramine…",989428
1474559,null,null,null,null,null,"""Tylenol""",300.0,"""1x/day""","""2 days""","""Acetaminophen""",1042329


measurement_id,department,HR,temp,lab_name,lab_value,medication,dose,frequency,duration,generic_name,event_id
u32,cat,f64,f64,str,f64,cat,f32,cat,cat,cat,u32
38675,null,null,null,"""SpO2""",50.0,null,null,null,null,null,926085
38676,null,null,null,"""SpO2""",50.0,null,null,null,null,null,322214


However, the "canonical" deep learning output pipeline doesn't leverage these columns at all, so its output is unchanged: 

In [18]:
!ls --color -lah processed/unihpf_sample/DL_reps/

total 25M
drwxrwxr-x 2 mmd mmd 4.0K Aug 23 14:03 .
drwxrwxr-x 5 mmd mmd 4.0K Aug 23 14:03 ..
-rw-rw-r-- 1 mmd mmd 2.9M Aug 23 14:03 held_out_0.parquet
-rw-rw-r-- 1 mmd mmd  19M Aug 23 14:03 train_0.parquet
-rw-rw-r-- 1 mmd mmd 2.5M Aug 23 14:03 tuning_0.parquet


In [19]:
df = pl.scan_parquet('processed/unihpf_sample/DL_reps/tuning_*.parquet')
print("DL Dataframe Columns:\n  * " + '\n  * '.join(df.columns))
display(df.head(4).collect())

DL Dataframe Columns:
  * subject_id
  * static_measurement_indices
  * static_indices
  * start_time
  * time
  * dynamic_measurement_indices
  * dynamic_indices
  * dynamic_values


subject_id,static_measurement_indices,static_indices,start_time,time,dynamic_measurement_indices,dynamic_indices,dynamic_values
u8,list[u8],list[u8],datetime[μs],list[f64],list[list[u8]],list[list[u8]],list[list[f64]]
1,[5],[15],2010-02-12 20:16:13,"[0.0, 26.4, … 404190.8]","[[1, 3, … 8], [1, 3, 6], … [1, 3, 6]]","[[4, 9, … 26], [1, 9, 19], … [1, 9, 19]]","[[null, 24.831881, … 103.300003], [null, 24.831931, 50.0], … [null, 25.600363, 4.64]]"
5,[5],[15],2010-01-16 07:34:43,"[0.0, 4.766667, … 443795.6]","[[1, 3, … 8], [1, 3, 6], … [1, 3, 6]]","[[4, 9, … 26], [1, 9, 19], … [1, 9, 19]]","[[null, 38.637415, … 97.5], [null, 38.637424, 4.29], … [null, 39.481197, 52.0]]"
9,[5],[15],2010-05-25 03:00:54,"[0.0, 31.65, … 258334.966667]","[[1, 3, … 8], [1, 3, 6], … [1, 3, 6]]","[[4, 9, … 26], [1, 9, 19], … [1, 9, 19]]","[[null, 32.972281, … 96.099998], [null, 32.972341, 0.07], … [null, 33.463449, 51.0]]"
12,[5],[17],2010-02-06 13:42:56,"[0.0, 11.45, … 413201.083333]","[[1, 3, … 8], [1, 3, … 8], … [1, 3, 6]]","[[4, 9, … 26], [2, 9, … 26], … [1, 9, 19]]","[[null, 24.653173, … 98.099998], [null, 24.653195, … 99.5], … [null, 25.438786, 50.0]]"


## Converting to a UniHPF friendly DL representation

Let's convert this to something that permits a more natural UniHPF style representation.

We'll make our target output schema the (current) planned standardized HF dataset schema for these style of datasets:

```python
import pyarrow as pa
measurement = pa.struct([
    ('code', pa.string()),
    ('text_value', pa.string()),
    ('numeric_value', pa.float32()),
    ('datetime_value', pa.timestamp('us')),
])


event_nested = pa.struct([
    ('time', pa.timestamp('us')),
    ('measurements', pa.list_(measurement))
])


schema_nested = pa.schema([
    ('subject_id', pa.int64()),
    ('static_measurements', pa.list_(measurement)),
    ('events', pa.list_(event_nested)), # Require ordered by time
])
```


To do so, we'll modify the existing DL conversion code in `EventStream.data.dataset_polars`, which is reproduced below:
```python
def _melt_df(self, source_df: DF_T, id_cols: Sequence[str], measures: list[str]) -> pl.Expr:
    """Re-formats `source_df` into the desired deep-learning output format."""
    struct_exprs = []
    total_vocab_size = self.vocabulary_config.total_vocab_size
    idx_dt = self.get_smallest_valid_int_type(total_vocab_size)

    for m in measures:
        if m == "event_type":
            cfg = None
            modality = DataModality.SINGLE_LABEL_CLASSIFICATION
        else:
            cfg = self.measurement_configs[m]
            modality = cfg.modality

        if m in self.measurement_vocabs:
            idx_present_expr = pl.col(m).is_not_null() & pl.col(m).is_in(self.measurement_vocabs[m])
            idx_value_expr = pl.col(m).map_dict(self.unified_vocabulary_idxmap[m], return_dtype=idx_dt)
        else:
            idx_present_expr = pl.col(m).is_not_null()
            idx_value_expr = pl.lit(self.unified_vocabulary_idxmap[m][m]).cast(idx_dt)

        idx_present_expr = idx_present_expr.cast(pl.Boolean).alias("present")
        idx_value_expr = idx_value_expr.alias("index")

        if (modality == DataModality.UNIVARIATE_REGRESSION) and (
            cfg.measurement_metadata.value_type
            in (NumericDataModalitySubtype.FLOAT, NumericDataModalitySubtype.INTEGER)
        ):
            val_expr = pl.col(m)
        elif modality == DataModality.MULTIVARIATE_REGRESSION:
            val_expr = pl.col(cfg.values_column)
        else:
            val_expr = pl.lit(None).cast(pl.Float64)

        struct_exprs.append(
            pl.struct([idx_present_expr, idx_value_expr, val_expr.alias("value")]).alias(m)
        )

    measurements_idx_dt = self.get_smallest_valid_int_type(len(self.unified_measurements_idxmap))
    return (
        source_df.select(*id_cols, *struct_exprs)
        .melt(
            id_vars=id_cols,
            value_vars=measures,
            variable_name="measurement",
            value_name="value",
        )
        .filter(pl.col("value").struct.field("present"))
        .select(
            *id_cols,
            pl.col("measurement")
            .map_dict(self.unified_measurements_idxmap)
            .cast(measurements_idx_dt)
            .alias("measurement_index"),
            pl.col("value").struct.field("index").alias("index"),
            pl.col("value").struct.field("value").alias("value"),
        )
    )

def build_DL_cached_representation(
    self, subject_ids: list[int] | None = None, do_sort_outputs: bool = False
) -> DF_T:
    # Identify the measurements sourced from each dataframe:
    subject_measures, event_measures, dynamic_measures = [], ["event_type"], []
    for m in self.unified_measurements_vocab[1:]:
        temporality = self.measurement_configs[m].temporality
        match temporality:
            case TemporalityType.STATIC:
                subject_measures.append(m)
            case TemporalityType.FUNCTIONAL_TIME_DEPENDENT:
                event_measures.append(m)
            case TemporalityType.DYNAMIC:
                dynamic_measures.append(m)
            case _:
                raise ValueError(f"Unknown temporality type {temporality} for {m}")

    # 1. Process subject data into the right format.
    if subject_ids:
        subjects_df = self._filter_col_inclusion(self.subjects_df, {"subject_id": subject_ids})
    else:
        subjects_df = self.subjects_df

    static_data = (
        self._melt_df(subjects_df, ["subject_id"], subject_measures)
        .groupby("subject_id")
        .agg(
            pl.col("measurement_index").alias("static_measurement_indices"),
            pl.col("index").alias("static_indices"),
        )
    )

    # 2. Process event data into the right format.
    if subject_ids:
        events_df = self._filter_col_inclusion(self.events_df, {"subject_id": subject_ids})
        event_ids = list(events_df["event_id"])
    else:
        events_df = self.events_df
        event_ids = None
    event_data = self._melt_df(events_df, ["subject_id", "timestamp", "event_id"], event_measures)

    # 3. Process measurement data into the right base format:
    if event_ids:
        dynamic_measurements_df = self._filter_col_inclusion(
            self.dynamic_measurements_df, {"event_id": event_ids}
        )
    else:
        dynamic_measurements_df = self.dynamic_measurements_df

    dynamic_ids = ["event_id", "measurement_id"] if do_sort_outputs else ["event_id"]
    dynamic_data = self._melt_df(dynamic_measurements_df, dynamic_ids, dynamic_measures)

    if do_sort_outputs:
        dynamic_data = dynamic_data.sort("event_id", "measurement_id")

    # 4. Join dynamic and event data.

    event_data = pl.concat([event_data, dynamic_data], how="diagonal")
    event_data = (
        event_data.groupby("event_id")
        .agg(
            pl.col("timestamp").drop_nulls().first().alias("timestamp"),
            pl.col("subject_id").drop_nulls().first().alias("subject_id"),
            pl.col("measurement_index").alias("dynamic_measurement_indices"),
            pl.col("index").alias("dynamic_indices"),
            pl.col("value").alias("dynamic_values"),
        )
        .sort("subject_id", "timestamp")
        .groupby("subject_id")
        .agg(
            pl.col("timestamp").first().alias("start_time"),
            ((pl.col("timestamp") - pl.col("timestamp").min()).dt.nanoseconds() / (1e9 * 60)).alias(
                "time"
            ),
            pl.col("dynamic_measurement_indices"),
            pl.col("dynamic_indices"),
            pl.col("dynamic_values"),
        )
    )

    out = static_data.join(event_data, on="subject_id", how="outer")
    if do_sort_outputs:
        out = out.sort("subject_id")

    return out
```

Our new code will be very similar, but it will 

  1. Capture the extra output columns when present
  2. Use the nested struct format instead of aligned lists
  3. Use string types instead of categorical types

In [20]:
from typing import Sequence
from EventStream.data.types import DataModality, TemporalityType, NumericDataModalitySubtype

def _HF_template_melt_df(
    self, source_df: pl.DataFrame, id_cols: Sequence[str], measures: list[str],
    default_struct_fields: dict[str, pl.DataType] | None = None,
) -> pl.Expr:
    """Re-formats `source_df` into the desired deep-learning output format."""
    struct_fields_by_m = {}
    total_vocab_size = self.vocabulary_config.total_vocab_size
    idx_dt = self.get_smallest_valid_int_type(total_vocab_size)
    
    if default_struct_fields is None: default_struct_fields = {}
    else: default_struct_fields = {**default_struct_fields}

    for m in measures:
        if m == "event_type":
            cfg = None
            modality = DataModality.SINGLE_LABEL_CLASSIFICATION
        else:
            cfg = self.measurement_configs[m]
            modality = cfg.modality

        if modality != DataModality.UNIVARIATE_REGRESSION:
            idx_value_expr = (
                pl.when(pl.col(m).is_not_null())
                .then(f"{m}/" + pl.col(m).cast(pl.Utf8))
                .otherwise(pl.lit(None, dtype=pl.Utf8))
            )
        else:
            idx_value_expr = (
                pl.when(pl.col(m).is_not_null())
                .then(pl.lit(f"{m}", dtype=pl.Utf8))
                .otherwise(pl.lit(None, dtype=pl.Utf8))
            )
            
        idx_value_expr = idx_value_expr.alias("code")

        if (modality == DataModality.UNIVARIATE_REGRESSION) and (
            cfg.measurement_metadata.value_type
            in (NumericDataModalitySubtype.FLOAT, NumericDataModalitySubtype.INTEGER)
        ):
            val_expr = pl.col(m).cast(pl.Float32)
        elif modality == DataModality.MULTIVARIATE_REGRESSION:
            val_expr = pl.col(cfg.values_column).cast(pl.Float32)
        else:
            val_expr = pl.lit(None, dtype=pl.Float32)
            
        struct_fields = {**default_struct_fields}
        
        struct_fields.update({
            'code': idx_value_expr,
            'numeric_value': val_expr.alias("numeric_value"),
        })
                
        if cfg is not None and cfg.modifiers is not None:
            for mod_col in cfg.modifiers:
                mod_col_expr = pl.col(mod_col)
                if source_df[mod_col].dtype == pl.Categorical:
                    mod_col_expr = mod_col_expr.cast(pl.Utf8)
                    
                struct_fields[mod_col] = mod_col_expr.alias(mod_col)
        
        struct_fields_by_m[m] = struct_fields
        
    struct_field_order = ['code', 'numeric_value', 'text_value', 'datetime_value']
    struct_field_order += sorted([k for k in default_struct_fields.keys() if k not in struct_field_order])
    struct_exprs = [
        pl.struct([fields[k] for k in struct_field_order]).alias(m)
        for m, fields in struct_fields_by_m.items()
    ]
    
    return (
        source_df.select(*id_cols, *struct_exprs)
        .melt(
            id_vars=id_cols,
            value_vars=measures,
            variable_name="_to_drop",
            value_name="measurement",
        )
        .filter(pl.col("measurement").struct.field("code").is_not_null())
        .select(*id_cols, "measurement")
    )


def build_HF_representation(
    self, subject_ids: list[int] | None = None, do_sort_outputs: bool = False
) -> pl.DataFrame:
    # Identify the measurements sourced from each dataframe:
    subject_measures, event_measures, dynamic_measures = [], ["event_type"], []
    default_struct_fields = {
        'text_value': pl.lit(None, dtype=pl.Utf8).alias('text_value'),
        'datetime_value': pl.lit(None, dtype=pl.Datetime).alias("datetime_value"),
    }
    for m in self.unified_measurements_vocab[1:]:
        cfg = self.measurement_configs[m]
        match cfg.temporality:
            case TemporalityType.STATIC:
                source_df = self.subjects_df
                subject_measures.append(m)
            case TemporalityType.FUNCTIONAL_TIME_DEPENDENT:
                source_df = self.events_df
                event_measures.append(m)
            case TemporalityType.DYNAMIC:
                source_df = self.dynamic_measurements_df
                dynamic_measures.append(m)
            case _:
                raise ValueError(f"Unknown temporality type {cfg.temporality} for {m}")
        
        if cfg.modifiers is None: continue
            
        for mod_col in cfg.modifiers:
            if mod_col not in source_df:
                raise IndexError(f"mod_col {mod_col} missing!")
            
            out_dt = source_df[mod_col].dtype
            if out_dt == pl.Categorical:
                out_dt = pl.Utf8
            default_struct_fields[mod_col] = pl.lit(None, dtype=out_dt).alias(mod_col)

    # 1. Process subject data into the right format.
    if subject_ids:
        subjects_df = self._filter_col_inclusion(self.subjects_df, {"subject_id": subject_ids})
    else:
        subjects_df = self.subjects_df
        
    static_data = (
        _HF_template_melt_df(
            self, subjects_df, ["subject_id"], subject_measures,
            default_struct_fields=default_struct_fields
        )
        .groupby("subject_id")
        .agg(pl.col("measurement").alias("static_measurements"))
    )

    # 2. Process event data into the right format.
    if subject_ids:
        events_df = self._filter_col_inclusion(self.events_df, {"subject_id": subject_ids})
        event_ids = list(events_df["event_id"])
    else:
        events_df = self.events_df
        event_ids = None
    event_data = _HF_template_melt_df(
        self, events_df, ["subject_id", "timestamp", "event_id"], event_measures,
        default_struct_fields=default_struct_fields
    )

    # 3. Process measurement data into the right base format:
    if event_ids:
        dynamic_measurements_df = self._filter_col_inclusion(
            self.dynamic_measurements_df, {"event_id": event_ids}
        )
    else:
        dynamic_measurements_df = self.dynamic_measurements_df

    dynamic_ids = ["event_id", "measurement_id"] if do_sort_outputs else ["event_id"]
    dynamic_data = _HF_template_melt_df(
        self, dynamic_measurements_df, dynamic_ids, dynamic_measures,
        default_struct_fields=default_struct_fields
    )

    if do_sort_outputs:
        dynamic_data = dynamic_data.sort("event_id", "measurement_id")

    # 4. Join dynamic and event data.

    event_data = pl.concat([event_data, dynamic_data], how="diagonal")
    event_data = (
        event_data.groupby("event_id")
        .agg(
            pl.col('subject_id').drop_nulls().first(),
            pl.col('timestamp').drop_nulls().first(),
            pl.col("measurement").alias("measurements")
        )
        .with_columns(
            pl.struct([
                pl.col("timestamp").alias("time"),
                pl.col("measurements").alias("measurements")
            ]).alias("event")
        )
        .sort("subject_id", "timestamp")
        .groupby("subject_id")
        .agg(pl.col("event").alias("events"))
    )

    out = static_data.join(event_data, on="subject_id", how="outer")
    if do_sort_outputs:
        out = out.sort("subject_id")

    return out

In [22]:
import pyarrow as pa
measurement = pa.struct([
    ('code', pa.string()),
    ('numeric_value', pa.float32()),
    ('text_value', pa.string()),
    ('datetime_value', pa.timestamp('us')),
   
    # Extra fields -- only needed here as we explicitly convert from polars to pyarrow format
    ('dose', pa.float32()),
    ('duration', pa.string()),
    ('frequency', pa.string()),
    ('generic_name', pa.string()),
])


event = pa.struct([
    ('time', pa.timestamp('us')),
    ('measurements', pa.list_(measurement))
])


schema = pa.schema([
    ('subject_id', pa.int64()),
    ('static_measurements', pa.list_(measurement)),
    ('events', pa.list_(event)), # Require ordered by time
])

In [23]:
import math, numpy as np
import pyarrow.parquet
from tqdm.auto import tqdm

for sp, subjs in tqdm(list(ESD.split_subjects.items())):
    n_chunks = int(math.ceil(len(subjs) / 20))
    for i, subjs_chunk in enumerate(np.array_split(list(subjs), n_chunks)):
        df = build_HF_representation(ESD, do_sort_outputs=True, subject_ids=list(subjs_chunk))
        arr_table = df.to_arrow().cast(schema)
        fp = ESD.config.save_dir / "HF_Dataset" / sp / f"{i}.parquet"
        fp.parent.mkdir(exist_ok=True, parents=True)
        pyarrow.parquet.write_table(arr_table, fp)

  0%|          | 0/3 [00:00<?, ?it/s]

Now that we've run this, what does out output look like?

In [24]:
print("In raw form:")
display(df.head(2))
print("Exploded out:")
display(
    df[0].explode('events').unnest('events')
    .explode('measurements').unnest('measurements')
    .filter(pl.col('code').str.starts_with('lab_name'))
    .head(2)
)
display(
    df[0].explode('events').unnest('events')
    .explode('measurements').unnest('measurements')
    .filter(pl.col('code').str.starts_with('medication'))
    .head(2)
)

In raw form:


subject_id,static_measurements,events
u8,list[struct[8]],list[struct[2]]
1,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]","[{2010-02-12 20:16:13,[{""event_type/ADMISSION&VITAL"",null,null,null,null,null,null,null}, {""age"",24.831881,null,null,null,null,null,null}, … {""temp"",103.300003,null,null,null,null,null,null}]}, {2010-02-12 20:42:37,[{""event_type/LAB"",null,null,null,null,null,null,null}, {""age"",24.831932,null,null,null,null,null,null}, {""lab_name/SpO2"",50.0,null,null,null,null,null,null}]}, … {2010-11-20 12:47:01,[{""event_type/LAB"",null,null,null,null,null,null,null}, {""age"",25.600363,null,null,null,null,null,null}, {""lab_name/potassium"",4.64,null,null,null,null,null,null}]}]"
5,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]","[{2010-01-16 07:34:43,[{""event_type/ADMISSION&VITAL"",null,null,null,null,null,null,null}, {""age"",38.637413,null,null,null,null,null,null}, … {""temp"",97.5,null,null,null,null,null,null}]}, {2010-01-16 07:39:29,[{""event_type/LAB"",null,null,null,null,null,null,null}, {""age"",38.637424,null,null,null,null,null,null}, {""lab_name/potassium"",4.29,null,null,null,null,null,null}]}, … {2010-11-20 12:10:19,[{""event_type/LAB"",null,null,null,null,null,null,null}, {""age"",39.481197,null,null,null,null,null,null}, {""lab_name/SpO2"",52.0,null,null,null,null,null,null}]}]"


Exploded out:


subject_id,static_measurements,time,code,numeric_value,text_value,datetime_value,dose,duration,frequency,generic_name
u8,list[struct[8]],datetime[μs],str,f32,str,datetime[μs],f32,str,str,str
1,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]",2010-02-12 20:42:37,"""lab_name/SpO2""",50.0,null,null,null,null,null,null
1,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]",2010-02-12 22:03:44,"""lab_name/SpO2""",51.0,null,null,null,null,null,null


subject_id,static_measurements,time,code,numeric_value,text_value,datetime_value,dose,duration,frequency,generic_name
u8,list[struct[8]],datetime[μs],str,f32,str,datetime[μs],f32,str,str,str
1,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]",2010-02-18 00:08:00,"""medication/mot…",null,null,null,700.0,"""6 days""","""1x/day""","""Ibuprofen"""
1,"[{""eye_color/BROWN"",null,null,null,null,null,null,null}]",2010-02-20 12:35:09,"""medication/Adv…",null,null,null,600.0,"""4 days""","""2x/day""","""Ibuprofen"""


We can use this as the specification for a Huggingface Dataset which can then be processed for tokenization, subsequence sampling, etc.

In [26]:
import datasets

features = datasets.Features.from_arrow_schema(schema)
ds = datasets.load_dataset(
    "parquet",
    data_files={
        sp: str(ESD.config.save_dir / "HF_Dataset" / sp / "*.parquet")
        for sp in ESD.split_subjects
    },
    features=features
)

print(ds['train'][0])

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating held_out split: 0 examples [00:00, ? examples/s]

Generating tuning split: 0 examples [00:00, ? examples/s]

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
ds['train'][0]

{'subject_id': 0,
 'static_measurements': [{'code': 'eye_color/GREEN',
   'numeric_value': None,
   'text_value': None,
   'datetime_value': None,
   'dose': None,
   'duration': None,
   'frequency': None,
   'generic_name': None}],
 'events': [{'time': datetime.datetime(2010, 6, 24, 13, 23),
   'measurements': [{'code': 'event_type/ADMISSION&VITAL',
     'numeric_value': None,
     'text_value': None,
     'datetime_value': None,
     'dose': None,
     'duration': None,
     'frequency': None,
     'generic_name': None},
    {'code': 'age',
     'numeric_value': 28.90775489807129,
     'text_value': None,
     'datetime_value': None,
     'dose': None,
     'duration': None,
     'frequency': None,
     'generic_name': None},
    {'code': 'department/ORTHOPEDIC',
     'numeric_value': None,
     'text_value': None,
     'datetime_value': None,
     'dose': None,
     'duration': None,
     'frequency': None,
     'generic_name': None},
    {'code': 'HR',
     'numeric_value': 125.80